## YOLO V3 Model

## Environment Setup

In [ ]:
# !pip install --upgrade tensorflow
# !mkdir data
# !wget wget https://pjreddie.com/media/files/yolov3.weights -O data/yolov3.weights
# !wget https://pjreddie.com/media/files/yolov3-tiny.weights -O data/yolov3-tiny.weights

## Library Imports

In [1]:
import cv2, datetime, imutils, math
import numpy as np
from itertools import combinations
from scipy.spatial import distance
from centroidtracker import CentroidTracker

CONFIDENCE_THRESH = 0.5 # Seuil minimal de detection d'une personne
nms_thresh = 0.5 # Seuil minimal de detection
MIN_DISTANCE = 100 # ?
VIDEO_RECORD_SPEED = 18.0

classes_file = "coco.names" # fichier texte contenant chaque classe d'objet détecté
weights = "yolov3.weights" # poids des paramètres du model pré-entraîné
config_file = "yolov3.cfg" # Ensemble des paramètres du model pré-entraînés

In [2]:
# Chargement de yolo v3 en utilisant cv2.dnn.readNet en passant les poids et les paramètres
net = cv2.dnn.readNet(weights, config_file)

# Lecture des noms des classes à partir du fichier texte "coco.names"
classes = []
with open(classes_file, "r") as f:
    classes = f.read().splitlines()
    
# exécution des inférences sur le réseau et collection des prédictions à partir des couches de sortie
# output_layers_names = net.getUnconnectedOutLayersNames()
# layerOutputs = net.forward(output_layers_names)


# défintion des couches de sortie car c'est là que nous définirons quel objet est détecté
layer_names = net.getLayerNames()
layerOutputs = [layer_names [i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors= np.random.uniform(0,255,size=(len(classes),3))

In [3]:
# Ensuite, chargeons une image. Nous réduirons la hauteur et la largeur de notre image à une échelle 
# de 40% et 30%. Et enregistrez toutes ces valeurs dans les variables de hauteur, largeur, canaux pour 
# l'image d'origine.

frame = cv2.imread("ab.jpg")
frame = cv2.resize(frame, None, fx=0.8, fy=0.6)
height, width, channels = frame.shape


# cv2.imshow("Predited_Image", frame)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
    
# nous définirons les couches de sortie car c'est là que nous définirons quel objet est 
# détecté en utilisant net.getUnconnectedOutLayers et net.getLayerNames

# nous ne pouvons pas donner cette image directement à l'algorithme, nous devons donc effectuer 
# une conversion à partir de cette image appelée conversion blob qui consiste 
# essentiellement à extraire des fonctionnalités de l'image

# Détection des objets dans le blob en utilisant cv2.dnn.blobFromImage et en passant 
# quelques arguments: frame: la capture, 1/255.0: facteur d'échelle, taille de l'image 
# à utiliser dans le blob être (416,416), pas de soustraction moyenne des calques comme (0,0,0), 
# swapRB=True: signifie que nous inverserons le bleu avec le rouge car OpenCV utilise BGR 
# mais nous avons des canaux dans l'image en RVB.

blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), (0,0,0), swapRB=True, crop=False)


# Voyons maintenant à quoi ressemblent les 3 objets blob différents en utilisant le code suivant. 
# Nous n'observons pas beaucoup de différence mais c'est ce que nous allons entrer dans l'algorithme YOLO.

# for b in blob: 
#     for n, img_blob in enumerate(b): 
#         cv2.imshow(str(n), img_blob)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()


# Nous transmettons maintenant cet objet blob au réseau en utilisant net.setInput (blob) , puis le 
# transmettons aux couches de sortie. Ici, tous les objets ont été détectés et outs contient toutes 
# les informations dont nous avons besoin pour extraire la position de l'objet comme les positions en haut, 
# à gauche, à droite, en bas, le nom de la classe.

net.setInput(blob)
outs = net.forward(layerOutputs)
#     print(outs[1])


'''
Évaluons maintenant les sorties en affichant les informations à l'écran. Nous essaierons principalement
de prédire la confiance, ce qui signifie à quel point l'algorithme est confiant lorsqu'il prévoit un objet. 
Pour cela, nous bouclerons les outs, obtenons d'abord tous les scores de chacun des outs . Ensuite, récupérez 
le class_id qui a le score le plus élevé parmi eux, puis attribuez une confiance à la valeur des scores en 
passant class_id .
Nous allons maintenant attribuer le seuil du niveau de confiance à 0,5. Tout ce qui est supérieur à 0,5 
devrait signifier un objet détecté. Soit aussi center_x, center_y, w comme largeur, h comme hauteur de 
l'objet détecté. Ici, nous allons utiliser les variables de hauteur et de largeur que nous avons enregistrées 
précédemment de l'image originale.
En outre , dessinons un rectangle autour de l'objet détecté en utilisant center_x, center_y, w, h . 
Et ajoutez quelques informations à cela comme la classe, la confiance.
'''

# affichage des informations à l'écran / obtenir le score de confiance de l'algorithme dans la détection 
# d'un objet dans le blob


# pour chaque détection de chaque couche de sortie, obtenez la confiance, l'ID de classe, les paramètres 
# de la boîte englobante et ignorez les détections faibles (confiance <0,5) pour chaque détection de chaque 
# couche de sortie, obtenir les paramètres de confiance, d'identification de classe, de boîte englobante et 
# ignorer les détections faibles (confiance < 0,5)


class_ids=[]
confidences=[]
boxes=[]
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            #onject detected
            center_x= int(detection[0]*width)
            center_y= int(detection[1]*height)
            w = int(detection[2]*width)
            h = int(detection[3]*height)
        
            #cv2.circle(img,(center_x,center_y),10,(0,255,0),2)
            
            #rectangle co-ordinaters
            x=int(center_x - w/2)
            y=int(center_y - h/2)
            #cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
            
            boxes.append([x,y,w,h]) #put all rectangle areas
            confidences.append(float(confidence)) #how confidence was that object detected and show that percentage
            class_ids.append(class_id) #name of the object tha was detected
            

    '''
    
    Il peut y avoir des cas où le même objet peut être détecté plusieurs fois comme ci-dessous. Vous voyez 
    que deux boîtes sont détectées chacune pour l'ordinateur portable et le moniteur. 
    Pour éliminer cela, nous utiliserons la fonctionnalité de suppression non maximale (NMS) qui permettar
    d'éliminer les boîtes en utilisant une valeur de seuil de 0,6 et permet de conserver que la meilleure 
    de toutes les boîtes. Et la variable index gardera une trace de ces objets uniques détectés.Donc pas de
    détection multiple pour le même objet.
    
    Ensuite, en utilisant la boucle ci-dessous sur toutes les boîtes trouvées, si la boîte apparaît dans 
    les index, l'algorithme dessinera uniquement un rectangle avec une couleur spécifique, met le texte du 
    nom de sa classe dessus.

    '''
indexes = cv2.dnn.NMSBoxes(boxes,confidences,0.4,0.6)




FONT = cv2.FONT_HERSHEY_COMPLEX_SMALL
FONT_SCALE = 1.0
FONT_THICKNESS = 2
TEST_COLOR = (255, 255, 255)
label_color = (0, 0, 0)

for i in range(len(boxes)):
    if i in indexes:
        x,y,w,h = boxes[i]
        label = '{} {:.2f}%'.format(classes[class_ids[i]],  confidences[i]*100)
        color = colors[i]
        cv2.rectangle(frame,(x,y),(x+w,y+h),color,2)

        (label_width, label_height), baseline = cv2.getTextSize(label, FONT, FONT_SCALE, FONT_THICKNESS)
        label_patch = np.zeros((label_height + baseline, label_width, 3), np.uint8)
        cv2.rectangle(frame, (x,y+int(0.5*baseline)),(x+label_width,y-label_height), color, FONT_THICKNESS-3)
        cv2.putText(frame, label, (x,y), FONT, 1, TEST_COLOR, FONT_THICKNESS-1)
        
cv2.imshow("Predited_Image", frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Object detection in video

In [ ]:
import cv2, datetime, imutils, math
import numpy as np
from itertools import combinations
from scipy.spatial import distance
from centroidtracker import CentroidTracker

CONFIDENCE_THRESH = 0.5 # Seuil minimal de detection d'une personne
nms_thresh = 0.5 # Seuil minimal de detection
MIN_DISTANCE = 100 # ?
VIDEO_RECORD_SPEED = 18.0

classes_file = "coco.names" # fichier texte contenant chaque classe d'objet détecté
weights = "yolov3.weights" # poids des paramètres du model pré-entraîné
config_file = "yolov3.cfg" # Ensemble des paramètres du model pré-entraînés

In [ ]:
# Chargement de yolo v3 en utilisant cv2.dnn.readNet en passant les poids et les paramètres
net = cv2.dnn.readNet(weights, config_file)

# Lecture des noms des classes à partir du fichier texte "coco.names"
classes = []
with open(classes_file, "r") as f:
    classes = f.read().splitlines()
    
# exécution des inférences sur le réseau et collection des prédictions à partir des couches de sortie
# output_layers_names = net.getUnconnectedOutLayersNames()
# layerOutputs = net.forward(output_layers_names)


# défintion des couches de sortie car c'est là que nous définirons quel objet est détecté
layer_names = net.getLayerNames()
layerOutputs = [layer_names [i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors= np.random.uniform(0,255,size=(len(classes),3))

In [ ]:
cap = cv2.VideoCapture('market.mp4')

# Configuration de la sortie de VideoWriter
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS)) # VIDEO_RECORD_SPEED
codec = cv2.VideoWriter_fourcc(*"XVID") # MJPG
video_outp = cv2.VideoWriter("Sortie.avi", codec, fps, (width, height))

while True:
    
    dt = "AIMS-SENEGAL, " + "{}".format(datetime.datetime.now())
    font = cv2.FONT_HERSHEY_SIMPLEX

    _, frame = cap.read()
    
    # Ensuite, chargeons une image. Nous réduirons la hauteur et la largeur de notre image à une échelle 
    # de 40% et 30%. Et enregistrez toutes ces valeurs dans les variables de hauteur, largeur, canaux pour 
    # l'image d'origine.

    frame = cv2.resize(frame, None, fx=0.8, fy=0.6)
    height, width, channels = frame.shape
#     height, width = frame.shape[:2] 
    
   
    
    # nous ne pouvons pas donner cette image directement à l'algorithme, nous devons donc effectuer 
    # une conversion à partir de cette image appelée conversion blob qui consiste 
    # essentiellement à extraire des fonctionnalités de l'image
    
    # Détection des objets dans le blob en utilisant cv2.dnn.blobFromImage et en passant 
    # quelques arguments: frame: la capture, 1/255.0: facteur d'échelle, taille de l'image 
    # à utiliser dans le blob être (416,416), pas de soustraction moyenne des calques comme (0,0,0), 
    # swapRB=True: signifie que nous inverserons le bleu avec le rouge car OpenCV utilise BGR 
    # mais nous avons des canaux dans l'image en RVB.

    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), (0,0,0), swapRB=True, crop=False)


    # Voyons maintenant à quoi ressemblent les 3 objets blob différents en utilisant le code suivant. 
    # Nous n'observons pas beaucoup de différence mais c'est ce que nous allons entrer dans l'algorithme YOLO.

    # for b in blob: 
    #     for n, img_blob in enumerate(b): 
    #         cv2.imshow(str(n), img_blob)
    #     cv2.waitKey(0)
    #     cv2.destroyAllWindows()


    # Nous transmettons maintenant cet objet blob au réseau en utilisant net.setInput (blob) , puis le 
    # transmettons aux couches de sortie. Ici, tous les objets ont été détectés et outs contient toutes 
    # les informations dont nous avons besoin pour extraire la position de l'objet comme les positions en haut, 
    # à gauche, à droite, en bas, le nom de la classe.

    net.setInput(blob)
    outs = net.forward(layerOutputs)
    #     print(outs[1])


    '''
    Évaluons maintenant les sorties en affichant les informations à l'écran. Nous essaierons principalement
    de prédire la confiance, ce qui signifie à quel point l'algorithme est confiant lorsqu'il prévoit un objet. 
    Pour cela, nous bouclerons les outs, obtenons d'abord tous les scores de chacun des outs . Ensuite, récupérez 
    le class_id qui a le score le plus élevé parmi eux, puis attribuez une confiance à la valeur des scores en 
    passant class_id .
    
    Nous allons maintenant attribuer le seuil du niveau de confiance à 0,5. Tout ce qui est supérieur à 0,5 
    devrait signifier un objet détecté. Soit aussi center_x, center_y, w comme largeur, h comme hauteur de 
    l'objet détecté. Ici, nous allons utiliser les variables de hauteur et de largeur que nous avons enregistrées 
    précédemment de l'image originale.
    
    En outre , dessinons un rectangle autour de l'objet détecté en utilisant center_x, center_y, w, h . 
    Et ajoutez quelques informations à cela comme la classe, la confiance.
    '''

    # affichage des informations à l'écran / obtenir le score de confiance de l'algorithme dans la détection 
    # d'un objet dans le blob
    
    class_ids=[]
    confidences=[]
    boxes=[]

    # pour chaque détection de chaque couche de sortie, obtenez la confiance, l'ID de classe, les paramètres 
    # de la boîte englobante et ignorez les détections faibles (confiance <0,5) pour chaque détection de chaque 
    # couche de sortie, obtenir les paramètres de confiance, d'identification de classe, de boîte englobante et 
    # ignorer les détections faibles (confiance < 0,5)

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                #onject detected
                center_x= int(detection[0]*width)
                center_y= int(detection[1]*height)
                w = int(detection[2]*width)
                h = int(detection[3]*height)

                #rectangle co-ordinaters
                x=int(center_x - w/2)
                y=int(center_y - h/2)
                #cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)

                boxes.append([x,y,w,h]) # put all rectangle areas
                confidences.append(float(confidence)) #how confidence was that object detected and show that percentage
                class_ids.append(class_id) # name of the object tha was detected


        '''

        Il peut y avoir des cas où le même objet peut être détecté plusieurs fois comme ci-dessous. Vous voyez 
        que deux boîtes sont détectées chacune pour l'ordinateur portable et le moniteur. 
        Pour éliminer cela, nous utiliserons la fonctionnalité de suppression non maximale (NMS) qui permettar
        d'éliminer les boîtes en utilisant une valeur de seuil de 0,6 et permet de conserver que la meilleure 
        de toutes les boîtes. Et la variable index gardera une trace de ces objets uniques détectés.Donc pas de
        détection multiple pour le même objet.

        Ensuite, en utilisant la boucle ci-dessous sur toutes les boîtes trouvées, si la boîte apparaît dans 
        les index, l'algorithme dessinera uniquement un rectangle avec une couleur spécifique, met le texte du 
        nom de sa classe dessus.

        '''
    indexes = cv2.dnn.NMSBoxes(boxes,confidences,0.4,0.6)
#                 indexes = cv2.dnn.NMSBoxes(boxes, confidences, CONFIDENCE_THRESH, nms_thresh)

    FONT = cv2.FONT_HERSHEY_COMPLEX_SMALL
    FONT_SCALE = 1.0
    FONT_THICKNESS = 2
    TEST_COLOR = (255, 255, 255)
    label_color = (0, 0, 0)
    
    for i in indexes.flatten():
        x,y,w,h = boxes[i]
        label = '{} {:.2f}%'.format(classes[class_ids[i]],  confidences[i]*100)
        color = colors[class_ids[i]]
        cv2.rectangle(frame,(x,y),(x+w,y+h),color,2)

        (label_width, label_height), baseline = cv2.getTextSize(label, FONT, FONT_SCALE, FONT_THICKNESS)
        label_patch = np.zeros((label_height + baseline, label_width, 3), np.uint8)
        cv2.rectangle(frame, (x,y+int(0.5*baseline)),(x+label_width,y-label_height), color, FONT_THICKNESS-3)
        cv2.putText(frame, label, (x,y), FONT, 1, TEST_COLOR, FONT_THICKNESS-1)

        
    cv2.putText(frame, dt, (700, 710), font, 1, (210, 195, 155), 2, cv2.LINE_8)
    cv2.imshow('Video', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('p'):
        break
    video_outp.write(frame)
    
    
cap.release()
video_outp.release()
cv2.destroyAllWindows()

In [ ]:
cap = cv2.VideoCapture('test.mp4')

# Configuration de la sortie de VideoWriter
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS)) # VIDEO_RECORD_SPEED
codec = cv2.VideoWriter_fourcc(*"XVID") # MJPG
video_outp = cv2.VideoWriter("Sortie.avi", codec, fps, (frame_width, frame_height))

while True:
    
    dt = "AIMS-SENEGAL, " + "{}".format(datetime.datetime.now())
    font = cv2.FONT_HERSHEY_COMPLEX

    _, frame = cap.read()
    
    # Ensuite, chargeons une image. Nous réduirons la hauteur et la largeur de notre image à une échelle 
    # de 40% et 30%. Et enregistrez toutes ces valeurs dans les variables de hauteur, largeur, canaux pour 
    # l'image d'origine.

    frame = cv2.resize(frame, None, fx=0.6, fy=0.6)
    height, width, channels = frame.shape
#     height, width = frame.shape[:2] 
    
   
    
    # nous ne pouvons pas donner cette image directement à l'algorithme, nous devons donc effectuer 
    # une conversion à partir de cette image appelée conversion blob qui consiste 
    # essentiellement à extraire des fonctionnalités de l'image
    
    # Détection des objets dans le blob en utilisant cv2.dnn.blobFromImage et en passant 
    # quelques arguments: frame: la capture, 1/255.0: facteur d'échelle, taille de l'image 
    # à utiliser dans le blob être (416,416), pas de soustraction moyenne des calques comme (0,0,0), 
    # swapRB=True: signifie que nous inverserons le bleu avec le rouge car OpenCV utilise BGR 
    # mais nous avons des canaux dans l'image en RVB.

    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), (0,0,0), swapRB=True, crop=False)


    # Voyons maintenant à quoi ressemblent les 3 objets blob différents en utilisant le code suivant. 
    # Nous n'observons pas beaucoup de différence mais c'est ce que nous allons entrer dans l'algorithme YOLO.

    # for b in blob: 
    #     for n, img_blob in enumerate(b): 
    #         cv2.imshow(str(n), img_blob)
    #     cv2.waitKey(0)
    #     cv2.destroyAllWindows()


    # Nous transmettons maintenant cet objet blob au réseau en utilisant net.setInput (blob) , puis le 
    # transmettons aux couches de sortie. Ici, tous les objets ont été détectés et outs contient toutes 
    # les informations dont nous avons besoin pour extraire la position de l'objet comme les positions en haut, 
    # à gauche, à droite, en bas, le nom de la classe.

    net.setInput(blob)
    outs = net.forward(layerOutputs)
    #     print(outs[1])


    '''
    Évaluons maintenant les sorties en affichant les informations à l'écran. Nous essaierons principalement
    de prédire la confiance, ce qui signifie à quel point l'algorithme est confiant lorsqu'il prévoit un objet. 
    Pour cela, nous bouclerons les outs, obtenons d'abord tous les scores de chacun des outs . Ensuite, récupérez 
    le class_id qui a le score le plus élevé parmi eux, puis attribuez une confiance à la valeur des scores en 
    passant class_id .
    
    Nous allons maintenant attribuer le seuil du niveau de confiance à 0,5. Tout ce qui est supérieur à 0,5 
    devrait signifier un objet détecté. Soit aussi center_x, center_y, w comme largeur, h comme hauteur de 
    l'objet détecté. Ici, nous allons utiliser les variables de hauteur et de largeur que nous avons enregistrées 
    précédemment de l'image originale.
    
    En outre , dessinons un rectangle autour de l'objet détecté en utilisant center_x, center_y, w, h . 
    Et ajoutez quelques informations à cela comme la classe, la confiance.
    '''

    # affichage des informations à l'écran / obtenir le score de confiance de l'algorithme dans la détection 
    # d'un objet dans le blob
    
#     class_ids=[]
    confidences=[]
    boxes=[]

    # pour chaque détection de chaque couche de sortie, obtenez la confiance, l'ID de classe, les paramètres 
    # de la boîte englobante et ignorez les détections faibles (confiance <0,5) pour chaque détection de chaque 
    # couche de sortie, obtenir les paramètres de confiance, d'identification de classe, de boîte englobante et 
    # ignorer les détections faibles (confiance < 0,5)

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if class_id == 0 and confidence >= 0.5:
                #onject detected
                center_x= int(detection[0]*width)
                center_y= int(detection[1]*height)
                w = int(detection[2]*width)
                h = int(detection[3]*height)

                #rectangle co-ordinaters
                x=int(center_x - w/2)
                y=int(center_y - h/2)
                #cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)

                boxes.append([x,y,w,h]) # put all rectangle areas
                confidences.append(float(confidence)) #how confidence was that object detected and show that percentage
#                 class_ids.append(class_id) # name of the object tha was detected


        '''

        Il peut y avoir des cas où le même objet peut être détecté plusieurs fois comme ci-dessous. Vous voyez 
        que deux boîtes sont détectées chacune pour l'ordinateur portable et le moniteur. 
        Pour éliminer cela, nous utiliserons la fonctionnalité de suppression non maximale (NMS) qui permettar
        d'éliminer les boîtes en utilisant une valeur de seuil de 0,6 et permet de conserver que la meilleure 
        de toutes les boîtes. Et la variable index gardera une trace de ces objets uniques détectés.Donc pas de
        détection multiple pour le même objet.

        Ensuite, en utilisant la boucle ci-dessous sur toutes les boîtes trouvées, si la boîte apparaît dans 
        les index, l'algorithme dessinera uniquement un rectangle avec une couleur spécifique, met le texte du 
        nom de sa classe dessus.

        '''
    indexes = cv2.dnn.NMSBoxes(boxes,confidences,0.4,0.6)
#                 indexes = cv2.dnn.NMSBoxes(boxes, confidences, CONFIDENCE_THRESH, nms_thresh)



    centroid_dict = dict()
    FONT = cv2.FONT_HERSHEY_COMPLEX_SMALL
#     FONT_SCALE = 1.0
    FONT_THICKNESS = 2
#     TEST_COLOR = (255, 255, 255)
#     label_color = (0, 0, 0)
    
    for i in indexes.flatten():
        x,y,w,h = boxes[i]
#         label = '{} {:.2f}%'.format(classes[class_ids[i]],  confidences[i]*100)
#         color = colors[class_ids[i]]
        color = (255, 255, 0)
        cv2.rectangle(frame,(x,y),(x+w,y+h),color, FONT_THICKNESS)

#         (label_width, label_height), baseline = cv2.getTextSize(label, FONT, FONT_SCALE, FONT_THICKNESS)
#         label_patch = np.zeros((label_height + baseline, label_width, 3), np.uint8)
#         cv2.rectangle(frame, (x,y+int(0.5*baseline)),(x+label_width,y-label_height), color, FONT_THICKNESS-3)
#         cv2.putText(frame, label, (x,y), FONT, 1, TEST_COLOR, FONT_THICKNESS-1)
        centroid_dict[i] = (x+int(w/2.0), y+int(h/2.0), x, y, x+w, y+h)
        
        
    red_zone_list = []
    for (id1, p1), (id2, p2) in combinations(centroid_dict.items(), 2):
        dx, dy = p1[0] - p2[0], p1[1] - p2[1]
        distance = math.sqrt(dx **2 + dy **2)
        if distance < 100.0:
            if id1 not in red_zone_list:
                red_zone_list.append(id1)
            if id2 not in red_zone_list:
                red_zone_list.append(id2)
            
    # draw the total number of social distancing violations on the output frame
    text = "Social Distancing Violations: {}".format(len(red_zone_list))
    cv2.putText(frame, text, (10, int(0.1*frame.shape[0])), font, 0.85, (0, 0, 255), 2)
            
    for id, box in centroid_dict.items():
            if id in red_zone_list:
                cv2.rectangle(frame, (box[2], box[3]), (box[4], box[5]), (0, 0, 255), 2)
            else:
                continue
#                 cv2.rectangle(frame, (box[2], box[3]), (box[4], box[5]), (255, 255, 0), 2)
            
        
    cv2.putText(frame, dt, (700, 710), font, 1, (210, 195, 155), 2, cv2.LINE_8)
    cv2.imshow('Video', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('p'):
        break
    video_outp.write(frame)
    
    
cap.release()
video_outp.release()
cv2.destroyAllWindows()